In [60]:
%logstop
%logstart -rtq ~/.logs/dw.py append
import seaborn as sns
sns.set()

In [3]:
from static_grader import grader

# DW Miniproject
## Introduction

The objective of this miniproject is to exercise your ability to wrangle tabular data set and aggregate large data sets into meaningful summary statistics. We'll work with the same medical data used in the `pw` miniproject but leverage the power of Pandas to more efficiently represent and act on our data.

## Downloading the data

We first need to download the data we'll be using from Amazon S3:

In [3]:
!mkdir dw-data
!wget http://dataincubator-wqu.s3.amazonaws.com/dwdata/201701scripts_sample.csv.gz -nc -P ./dw-data/ # !wget is just used to download the url in front of it
!wget http://dataincubator-wqu.s3.amazonaws.com/dwdata/201606scripts_sample.csv.gz -nc -P ./dw-data/
!wget http://dataincubator-wqu.s3.amazonaws.com/dwdata/practices.csv.gz -nc -P ./dw-data/
!wget http://dataincubator-wqu.s3.amazonaws.com/dwdata/chem.csv.gz -nc -P ./dw-data/

mkdir: cannot create directory ‘dw-data’: File exists
File ‘./dw-data/201701scripts_sample.csv.gz’ already there; not retrieving.

File ‘./dw-data/201606scripts_sample.csv.gz’ already there; not retrieving.

File ‘./dw-data/practices.csv.gz’ already there; not retrieving.

File ‘./dw-data/chem.csv.gz’ already there; not retrieving.



## Loading the data

Similar to the `pw` miniproject, the first step is to read in the data. The data files are stored as compressed CSV files. You can load the data into a pandas `DataFrame` by making use of the `pd.read_csv` function to parse the data into a `DataFrame`. You may want to check the pandas documentation for parsing [CSV](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) files for reference.

For a description of the data set please, refer to the [`pw` miniproject](./pw.ipynb). **Note that all questions make use of the 2017 data only, except for Question 5 which makes use of both the 2017 and 2016 data.**

In [4]:
import pandas as pd
import numpy as np

In [5]:
# load the 2017 data
scripts = pd.read_csv('dw-data/201701scripts_sample.csv.gz')
scripts.head()

,practice,bnf_code,bnf_name,items,nic,act_cost,quantity
0,N85639,0106020C0,Bisacodyl_Tab E/C 5mg,1,0.39,0.47,12
1,N85639,0106040M0,Movicol Plain_Paed Pdr Sach 6.9g,1,4.38,4.07,30
2,N85639,0301011R0,Salbutamol_Inha 100mcg (200 D) CFF,1,1.50,1.40,1
3,N85639,0304010G0,Chlorphenamine Mal_Oral Soln 2mg/5ml,1,2.62,2.44,150
4,N85639,0401020K0,Diazepam_Tab 2mg,1,0.16,0.26,6


In [63]:
col_names=[ 'code', 'name', 'addr_1', 'addr_2', 'borough', 'village', 'post_code']
practices = pd.read_csv('dw-data/practices.csv.gz', names=col_names)              # Now we can see that the data we are using is not having a header for every row, so we will be providing them in default
practices.head()

,code,name,addr_1,addr_2,borough,village,post_code
0,A81001,THE DENSHAM SURGERY,THE HEALTH CENTRE,LAWSON STREET,STOCKTON ON TEES,CLEVELAND,TS18 1HU
1,A81002,QUEENS PARK MEDICAL CENTRE,QUEENS PARK MEDICAL CTR,FARRER STREET,STOCKTON ON TEES,CLEVELAND,TS18 2AW
2,A81003,VICTORIA MEDICAL PRACTICE,THE HEALTH CENTRE,VICTORIA ROAD,HARTLEPOOL,CLEVELAND,TS26 8DB
3,A81004,WOODLANDS ROAD SURGERY,6 WOODLANDS ROAD,NaN,MIDDLESBROUGH,CLEVELAND,TS1 3BE
4,A81005,SPRINGWOOD SURGERY,SPRINGWOOD SURGERY,RECTORY LANE,GUISBOROUGH,NaN,TS14 7DJ


In [5]:
chem = pd.read_csv('dw-data/chem.csv.gz')
chem.head()

,CHEM SUB,NAME
0,0101010A0,Alexitol Sodium
1,0101010B0,Almasilate
2,0101010C0,Aluminium Hydroxide
3,0101010D0,Aluminium Hydroxide With Magnesium
4,0101010E0,Hydrotalcite


Now that we've loaded in the data, let's first replicate our results from the `pw` miniproject. Note that we are now working with a larger data set so the answers will be different than in the `pw` miniproject even if the analysis is the same.

## Question 1: summary_statistics

In the `pw` miniproject we first calculated the total, mean, standard deviation, and quartile statistics of the `'items'`, `'quantity'`', `'nic'`, and `'act_cost'` fields. To do this we had to write some functions to calculate the statistics and apply the functions to our data structure. The DataFrame has a `describe` method that will calculate most (not all) of these things for us.

Submit the summary statistics to the grader as a list of tuples: `[('act_cost', (total, mean, std, q25, median, q75)), ...]`

In [49]:
summary_stats = [('items', (0,) * 6), ('quantity', (0,) * 6), ('nic', (0,) * 6), ('act_cost', (0,) * 6)]

In [65]:
totals = scripts[['items', 'nic', 'act_cost','quantity']].sum()    
totals
median = scripts[['items', 'nic', 'act_cost','quantity']].median()
median

items         2.00
nic          22.64
act_cost     21.22
quantity    100.00
dtype: float64

In [44]:
stats = scripts.describe().T
stats

,count,mean,std,min,25%,50%,75%,max
items,973193.0,9.133136,29.204198,1.00,1.00,2.00,6.00,2384.00
nic,973193.0,73.058915,188.070257,0.00,7.80,22.64,65.00,16320.00
act_cost,973193.0,67.986613,174.401703,0.04,7.33,21.22,60.67,15108.32
quantity,973193.0,741.329835,3665.426958,0.00,28.00,100.00,350.00,577720.00


In [72]:
stats = stats.assign(total = totals)
stats = stats.assign(median = median)
col = ['total', 'mean', 'std', '25%' ,'median' , '75%']
stats = stats[col]
stats


,total,mean,std,25%,median,75%
items,8.888304e+06,9.133136,29.204198,1.00,2.00,6.00
nic,7.110042e+07,73.058915,188.070257,7.80,22.64,65.00
act_cost,6.616410e+07,67.986613,174.401703,7.33,21.22,60.67
quantity,7.214570e+08,741.329835,3665.426958,28.00,100.00,350.00


In [57]:
for row in stats:
    print(row)

total
mean
std
25%
75%


In [73]:
summary_stats = [(row[0],row[1:]) for row in stats.itertuples()]

In [85]:
del sa

In [74]:
summary_stats

[('items', (8888304.0, 9.133135976111625, 29.204198282803603, 1.0, 2.0, 6.0)),
 ('nic',
  (71100424.84000002, 73.05891517920908, 188.070256906825, 7.8, 22.64, 65.0)),
 ('act_cost',
  (66164096.11999999,
   67.98661326170655,
   174.40170332301963,
   7.33,
   21.22,
   60.67)),
 ('quantity',
  (721457006.0, 741.3298348837282, 3665.426958467915, 28.0, 100.0, 350.0))]

In [75]:
grader.score.dw__summary_statistics(summary_stats)

Your score: 1.000


## Question 2: most_common_item

We can also easily compute summary statistics on groups within the data. In the `pw` miniproject we had to explicitly construct the groups based on the values of a particular field. Pandas will handle that for us via the `groupby` method. This process is [detailed in the Pandas documentation](https://pandas.pydata.org/pandas-docs/stable/groupby.html).

Use `groupby` to calculate the total number of items dispensed for each `'bnf_name'`. Find the item with the highest total and return the result as `[(bnf_name, total)]`.

In [6]:
bnf_group = scripts.groupby('bnf_name')



In [9]:
bnf_group = bnf_group['items'].sum()

In [10]:
bnf_group.max()
bnf_group.idxmax()

'Omeprazole_Cap E/C 20mg'

In [12]:
most_common_item = [(bnf_group.idxmax(), bnf_group.max())]

In [13]:
grader.score.dw__most_common_item(most_common_item)

Your score: 1.000


## Question 3: items_by_region

Now let's find the most common item by post code. The post code information is in the `practices` DataFrame, and we'll need to `merge` it into the `scripts` DataFrame. Pandas provides [extensive documentation](https://pandas.pydata.org/pandas-docs/stable/merging.html) with diagrammed examples on different methods and approaches for joining data. The `merge` method is only one of many possible options.

Return your results as a list of tuples `(post code, item name, amount dispensed as % of total)`. Sort your results ascending alphabetically by post code and take only results from the first 100 post codes.

**NOTE:** Some practices have multiple postal codes associated with them. Use the alphabetically first postal code. Note some postal codes may have multiple `'bnf_name'` with the same prescription rate for the maximum. In this case, take the alphabetically first `'bnf_name'` (as in the `pw` miniproject).

In [8]:
practice_no_duplicates = practices.sort_values('post_code').drop_duplicates(subset='code',keep='first') #drop_duplicates is used to remove the duplicates of the table, subset is the name of the column you want to remove duplicates from and keep is used to tell that which duplicate to be taken first, last or false
practice_no_duplicates.shape

(10843, 7)

In [9]:
practices['code'].nunique() #nunique will tell us that how many unique rows we have, in which each row has different code

10843

In [10]:
practices_unique = practices.sort_values('post_code').drop_duplicates()

SyntaxError: unexpected EOF while parsing (<ipython-input-10-afea592b2acd>, line 1)

In [11]:
result = scripts.merge(practice_no_duplicates, left_on='practice', right_on='code') #left_on means that compare the entry of the column of the left dataset to the merge it with the same entry of the column of the right_on
result.head()

,practice,bnf_code,bnf_name,items,nic,act_cost,quantity,code,name,addr_1,addr_2,borough,village,post_code
0,N85639,0106020C0,Bisacodyl_Tab E/C 5mg,1,0.39,0.47,12,N85639,GP OOH VCH,VICTORIA CENTRAL HOSPITAL,MILL LANE,WALLASEY,NaN,CH44 5UF
1,N85639,0106040M0,Movicol Plain_Paed Pdr Sach 6.9g,1,4.38,4.07,30,N85639,GP OOH VCH,VICTORIA CENTRAL HOSPITAL,MILL LANE,WALLASEY,NaN,CH44 5UF
2,N85639,0301011R0,Salbutamol_Inha 100mcg (200 D) CFF,1,1.50,1.40,1,N85639,GP OOH VCH,VICTORIA CENTRAL HOSPITAL,MILL LANE,WALLASEY,NaN,CH44 5UF
3,N85639,0304010G0,Chlorphenamine Mal_Oral Soln 2mg/5ml,1,2.62,2.44,150,N85639,GP OOH VCH,VICTORIA CENTRAL HOSPITAL,MILL LANE,WALLASEY,NaN,CH44 5UF
4,N85639,0401020K0,Diazepam_Tab 2mg,1,0.16,0.26,6,N85639,GP OOH VCH,VICTORIA CENTRAL HOSPITAL,MILL LANE,WALLASEY,NaN,CH44 5UF


In [12]:
total_item_bnf_name_post_code = result.groupby(['post_code','bnf_name'])['items'].sum().reset_index() #Without reset_index we will be having multiple index that is one for post_code and one for bnf_name but by reset_index() we are having index like 0,1,2,3....
total_item_bnf_name_post_code = total_item_bnf_name_post_code.set_index('post_code')                  # We are setting the index by using set_index() because we wantt o index this dataframe accroding to the post_code because we will be adding the total column in it we will be adding it according to the post_code and not by 0,1,2,3.... index
total_item_bnf_name_post_code

,bnf_name,items
post_code,,
B11 4BW,3m Health Care_Cavilon Durable Barrier C,7
B11 4BW,3m Health Care_Cavilon No Sting Barrier,2
B11 4BW,Abasaglar KwikPen_100u/ml 3ml Pf Pen,2
B11 4BW,Abidec_Dps,63
B11 4BW,Able Spacer + Sml/Med Mask,1
...,...,...
YO16 4LZ,Zydol SR_Tab 50mg,1
YO16 4LZ,easiGRIP 12cm x 1m Stkntte Elasctd Tublr,1
YO16 4LZ,iMEDicare SomaCorrect Xtra Vacuum Pump,1


In [13]:
total_item_post_code = result.groupby('post_code')['items'].sum()
total_item_post_code.head()

post_code
B11 4BW    22731
B12 9LP    17073
B18 7AL    20508
B21 9RY    31027
B23 6DJ    28011
Name: items, dtype: int64

In [14]:
total_item_bnf_name_post_code['total'] = total_item_post_code
total_item_bnf_name_post_code

,bnf_name,items,total
post_code,,,
B11 4BW,3m Health Care_Cavilon Durable Barrier C,7,22731
B11 4BW,3m Health Care_Cavilon No Sting Barrier,2,22731
B11 4BW,Abasaglar KwikPen_100u/ml 3ml Pf Pen,2,22731
B11 4BW,Abidec_Dps,63,22731
B11 4BW,Able Spacer + Sml/Med Mask,1,22731
...,...,...,...
YO16 4LZ,Zydol SR_Tab 50mg,1,53847
YO16 4LZ,easiGRIP 12cm x 1m Stkntte Elasctd Tublr,1,53847
YO16 4LZ,iMEDicare SomaCorrect Xtra Vacuum Pump,1,53847


In [15]:
total_item_bnf_name_post_code['ratio'] = total_item_bnf_name_post_code['items'] / total_item_bnf_name_post_code['total']
total_item_bnf_name_post_code

,bnf_name,items,total,ratio
post_code,,,,
B11 4BW,3m Health Care_Cavilon Durable Barrier C,7,22731,0.000308
B11 4BW,3m Health Care_Cavilon No Sting Barrier,2,22731,0.000088
B11 4BW,Abasaglar KwikPen_100u/ml 3ml Pf Pen,2,22731,0.000088
B11 4BW,Abidec_Dps,63,22731,0.002772
B11 4BW,Able Spacer + Sml/Med Mask,1,22731,0.000044
...,...,...,...,...
YO16 4LZ,Zydol SR_Tab 50mg,1,53847,0.000019
YO16 4LZ,easiGRIP 12cm x 1m Stkntte Elasctd Tublr,1,53847,0.000019
YO16 4LZ,iMEDicare SomaCorrect Xtra Vacuum Pump,1,53847,0.000019


In [ ]:
output = total_item_bnf_name_post_code.groupby('post_code').apply(lambda df: df.reset_index().drop('post_code',axis=1).nlargest(1,'ratio')) # Here we are using a drop function which will be droping the first post_code which is due to axis=1. nlargest is giving us the largest value of the column of the ratio. 

In [56]:
output = output.reset_index()[['post_code','bnf_name','ratio']].sort_values('post_code').head(100)

,post_code,bnf_name,ratio
0,B11 4BW,Salbutamol_Inha 100mcg (200 D) CFF,0.031059
1,B12 9LP,Paracet_Tab 500mg,0.024893
2,B18 7AL,Salbutamol_Inha 100mcg (200 D) CFF,0.027111
3,B21 9RY,Metformin HCl_Tab 500mg,0.033294
4,B23 6DJ,Lansoprazole_Cap 30mg (E/C Gran),0.021384


In [ ]:
#output = [("B11 4BW", "Salbutamol_Inha 100mcg (200 D) CFF", 0.0310589063)] * 100

In [57]:
grader.score.dw__items_by_region(output)

Your score: 1.000


## Question 4: script_anomalies

Drug abuse is a source of human and monetary costs in health care. A first step in identifying practitioners that enable drug abuse is to look for practices where commonly abused drugs are prescribed unusually often. Let's try to find practices that prescribe an unusually high amount of opioids. The opioids we'll look for are given in the list below.

In [18]:
opioids = ['morphine', 'oxycodone', 'methadone', 'fentanyl', 'pethidine', 'buprenorphine', 'propoxyphene', 'codeine']

These are generic names for drugs, not brand names. Generic drug names can be found using the `'bnf_code'` field in `scripts` along with the `chem` table. Use the list of opioids provided above along with these fields to make a new field in the `scripts` data that flags whether the row corresponds with a opioid prescription.

In [19]:
chem

,CHEM SUB,NAME
0,0101010A0,Alexitol Sodium
1,0101010B0,Almasilate
2,0101010C0,Aluminium Hydroxide
3,0101010D0,Aluminium Hydroxide With Magnesium
4,0101010E0,Hydrotalcite
...,...,...
3482,0102000AK,Eluxadoline
3483,0405010T0,Naltrexone/Bupropion
3484,0605010AF,Follitropin Delta
3485,0406000AH,Rolapitant


In [20]:
chem['is_opioid'] = chem['NAME'].apply(lambda chem_name:any([True for opioid in opioids if opioid in chem_name.lower()]))

In [21]:
scripts = scripts.merge(chem , left_on='bnf_code' , right_on='CHEM SUB', how='left') # We are using how because it will make us merge even those rows which has no match for 'bnf_code' and 'CHEM SUB'. Else we were having a merge in which we were only having merging if we have a matching 'bnf_code' and 'CHEM SUB' fields.

In [22]:
# As we are having some fields in scripts_is_opioids which are NaN cuz we didnt found a match for them, so we are filling those fields with false value.
scripts = scripts.fillna(False)
scripts.head()

,practice,bnf_code,bnf_name,items,nic,act_cost,quantity,CHEM SUB,NAME,is_opioid
0,N85639,0106020C0,Bisacodyl_Tab E/C 5mg,1,0.39,0.47,12,0106020C0,Bisacodyl,False
1,N85639,0106040M0,Movicol Plain_Paed Pdr Sach 6.9g,1,4.38,4.07,30,0106040M0,Macrogol 3350,False
2,N85639,0301011R0,Salbutamol_Inha 100mcg (200 D) CFF,1,1.50,1.40,1,0301011R0,Salbutamol,False
3,N85639,0304010G0,Chlorphenamine Mal_Oral Soln 2mg/5ml,1,2.62,2.44,150,0304010G0,Chlorphenamine Maleate,False
4,N85639,0401020K0,Diazepam_Tab 2mg,1,0.16,0.26,6,0401020K0,Diazepam,False


Now for each practice calculate the proportion of its prescriptions containing opioids.

**Hint:** Consider the following list: `[0, 1, 1, 0, 0, 0]`. What proportion of the entries are 1s? What is the mean value?

In [23]:
# Now we want to create a mean for all opioids that were prescribed by a single practice(or medical store)
opioids_per_practice = scripts.groupby('practice')['is_opioid'].mean()

How do these proportions compare to the overall opioid prescription rate? Subtract off the proportion of all prescriptions that are opioids from each practice's proportion.

In [13]:
# Now we will calculate the opioid proportion per pratice relative to opioid proportion accross all practices
opioids_overall_prop = scripts['is_opioid'].mean()

0.035772733339493434

In [24]:
relative_opioids_per_practice = opioids_per_practice - scripts['is_opioid'].mean()


Now that we know the difference between each practice's opioid prescription rate and the overall rate, we can identify which practices prescribe opioids at above average or below average rates. However, are the differences from the overall rate important or just random deviations? In other words, are the differences from the overall rate big or small?

To answer this question we have to quantify the difference we would typically expect between a given practice's opioid prescription rate and the overall rate. This quantity is called the **standard error**, and is related to the **standard deviation**, $\sigma$. The standard error in this case is

$$ \frac{\sigma}{\sqrt{n}} $$

where $n$ is the number of prescriptions each practice made. Calculate the standard error for each practice. Then divide `relative_opioids_per_practice` by the standard errors. We'll call the final result `opioid_scores`.

In [54]:
standard_error_per_practice = scripts['is_opioid'].std() / (scripts['practice'].value_counts() ** 0.5) # Bascially in the denominator we are just counting the number of prescriptions in the practice, instead of 'is_opioid' we can use any data value.
opioid_scores = relative_opioids_per_practice / standard_error_per_practice #These are z-scores
opioid_scores

A81005   -0.551807
A81007    1.540186
A81011    2.293528
A81012    1.373723
A81017    0.586611
            ...   
Y05570    1.392462
Y05583   -1.019213
Y05597   -0.817189
Y05660   -1.275607
Y05670   -0.667232
Length: 856, dtype: float64

In [37]:
#practice_names = practices.sort_values('name').drop_duplicates('code')[['code','name']] --> In this case we are having everything right except we are not having a index that will match the index of opioid_scores dataframe, so we must set_index to the common column between two dataframes that is code.
practice_names = practices.sort_values('name').drop_duplicates('code').set_index('code')[['name']]
practice_names

,name
code,
P87657,(IRLAM) SALFORD CARE CTRS MEDICAL PRACTI
Y05381,0-19 EAST CHESHIRE HEALTH VISITORS
Y04082,0-19 PUBLIC HEALTH SERVICE HARTLEPOOL
Y00364,1 TO 1 CENTRE
P87652,1/LOWER BROUGHTON MEDICAL PRACTICE
...,...
M83013,YOXALL
Y05235,YPS SSM SERVICE (REFRESH)
E84653,ZAIN MEDICAL CENTRE


In [30]:
num_scripts = scripts['practice'].value_counts()
num_scripts

N83028    2846
L83100    2799
D81043    2461
B81008    2398
B81026    2349
          ... 
Y03179       1
Y03472       1
C85617       1
Y05366       1
Y03010       1
Name: practice, Length: 856, dtype: int64

In [59]:
result = ( opioid_scores
          .to_frame()
          .rename({0: 'z-score'}, axis = 1)
          .assign(num_scripts=num_scripts) 
          .assign(name=practice_names['name'])
          .reset_index()
          .loc[:,['index','name','z-score','num_scripts']]
          .nlargest(100,'z-score')
          
         ) #to_frame is converting it into a data frame, rename is renaming the column anme to 'z-score' and assign is doing the task of df['num_scripts']=num_scripts

result

,index,name,z-score,num_scripts
714,Y01852,NATIONAL ENHANCED SERVICE,11.700972,7
754,Y03006,OUTREACH SERVICE NH / RH,7.342237,2
772,Y03668,BRISDOC HEALTHCARE SERVICES OOH,6.154320,60
261,G81703,H&R P C SPECIAL SCHEME,5.126073,36
819,Y04997,HMR BARDOC OOH,4.963767,321
...,...,...,...,...
160,D81024,THOMAS WALKER,1.697803,1378
11,A81033,OAKFIELD MEDICAL PRACTICE,1.694036,1056
564,P81684,DR DP CHARLES' PRACTICE,1.692085,1354
487,N81121,NORTHGATE VILLAGE SURGERY,1.686724,1305


The quantity we have calculated in `opioid_scores` is called a **z-score**:

$$ \frac{\bar{X} - \mu}{\sqrt{\sigma^2/n}} $$

Here $\bar{X}$ corresponds with the proportion for each practice, $\mu$ corresponds with the proportion across all practices, $\sigma^2$ corresponds with the variance of the proportion across all practices, and $n$ is the number of prescriptions made by each practice. Notice $\bar{X}$ and $n$ will be different for each practice, while $\mu$ and $\sigma$ are determined across all prescriptions, and so are the same for every z-score. The z-score is a useful statistical tool used for hypothesis testing, finding outliers, and comparing data about different types of objects or events.

Now that we've calculated this statistic, take the 100 practices with the largest z-score. Return your result as a list of tuples in the form `(practice_code, practice_name, z-score, number_of_scripts)`. Sort your tuples by z-score in descending order. Note that some practice codes will correspond with multiple names. In this case, use the first match when sorting names alphabetically.

In [ ]:
unique_practices = ...
anomalies = [("Y01852", "NATIONAL ENHANCED SERVICE", 11.6958178629, 7)] * 100

In [ ]:
who

In [60]:
grader.score.dw__script_anomalies(result)

Your score: 1.000


## Question 5: script_growth

Another way to identify anomalies is by comparing current data to historical data. In the case of identifying sites of drug abuse, we might compare a practice's current rate of opioid prescription to their rate 5 or 10 years ago. Unless the nature of the practice has changed, the profile of drugs they prescribe should be relatively stable. We might also want to identify trends through time for business reasons, identifying drugs that are gaining market share. That's what we'll do in this question.

We'll load in beneficiary data from 6 months earlier, June 2016, and calculate the growth rate in prescription rate from June 2016 to January 2017 for each `bnf_name`. The growth is defined as

$$
\text{growth rate} = \frac{x_1 - x_0}{x_0},
$$
where $x_1$ is the current value and $x_0$ is the previous value.


We'll return the 50 items with largest growth and the 50 items with the largest shrinkage (i.e. negative growth rate) as a list of tuples sorted by growth rate in descending order in the format `(script_name, growth_rate, raw_2016_count)`. You'll notice that many of the 50 fastest growing items have low counts of prescriptions in 2016. Filter out any items that were prescribed less than 50 times.

In [108]:
scripts16 = pd.read_csv('dw-data/201606scripts_sample.csv.gz')


,practice,bnf_code,bnf_name,items,nic,act_cost,quantity
0,N85638,0301011R0,Salamol_Inha 100mcg (200 D) CFF (Teva),2,2.92,2.73,2
1,N85638,0301011R0,Easyhaler_Salbutamol Sulf 200mcg (200D),1,6.63,6.15,1
2,N85638,0301020I0,Ipratrop Brom_Inh Soln 500mcg/2ml Ud,1,1.77,1.75,12
3,N85638,0301020I0,Ipratrop Brom_Inh Soln 250mcg/1ml Ud,1,4.47,4.15,20
4,N85638,0302000C0,Clenil Modulite_Inha 50mcg (200D),1,3.70,3.44,1


In [63]:
rate_2016 = scripts16['bnf_name'].value_counts()   #It is doing the same task of scripts16.groupby('bnf_name').value_counts()

In [64]:
rate_2017 = scripts['bnf_name'].value_counts()

<bound method NDFrame.head of GlucoRX FinePoint Needles Pen Inj Screw     1718
Sod Feredetate_Oral Soln 190mg/5ml S/F       844
3m Health Care_Cavilon Durable Barrier C     816
Prednisolone_Tab 5mg                         785
Fluclox Sod_Cap 500mg                        783
                                            ... 
Cutimed Sorbion Sach Border 25x15cm Woun       1
ActiLymph Class 2 Thigh Open Toe + Wide        1
Draina S Vision 100 300ml cut to 100mm W       1
Methadone HCl_Oral Conc 10mg/1ml S/F           1
Heparin Sod_Inj 1 000u/ml 5ml Amp              1
Name: bnf_name, Length: 13471, dtype: int64>

In [119]:
growth_rate = (rate_2017 - rate_2016) / rate_2016
growth_rate = growth_rate.dropna()
growth_rate

365 Non Adherent 10cm x 10cm Pfa Plas Fa    0.000000
365 Transpt Island 8.5cm x 15.5cm VP Adh    0.000000
3m Health Care_Cavilon Durable Barrier C   -0.010909
3m Health Care_Cavilon No Sting 1ml Barr   -0.034632
3m Health Care_Cavilon No Sting 3ml Barr   -0.104762
                                              ...   
nSpire PiKo-1 Stnd Range Peak Flow Meter   -0.500000
nSpire Pocket Peak Low Range Peak Flow M    1.000000
nSpire Pocket Peak Stnd Range Peak Flow     1.500000
oraNurse_Toothpaste Orig (1450ppm)          1.000000
palmdoc (Reagent)_Strips                    0.666667
Name: bnf_name, Length: 11587, dtype: float64

In [120]:
result = (
    growth_rate
    .to_frame()
    .assign(count=rate_2016)
    .dropna()
    .reset_index()
    .rename({'index':'bnf_name','bnf_name':'growth_rate'},axis=1)
    .query('count > 50')
    .sort_values('growth_rate',ascending=False)
)
result


,bnf_name,growth_rate,count
1467,Butec_Transdermal Patch 5mcg/hr,3.467742,62
1465,Butec_Transdermal Patch 10mcg/hr,3.000000,69
4415,Fostair NEXThaler_Inh 200mcg/6mcg (120D),1.430233,86
10205,Sytron_Elix 190mg/5ml S/F,1.280130,307
8498,Pneumococcal_Vac 0.5ml Vl (23 Valent),1.269430,193
...,...,...,...
5230,Hydroxyzine HCl_Oral Soln 10mg/5ml,-0.914894,94
8007,Ovysmen_Tab,-0.925373,67
1976,Climaval_Tab 1mg,-0.926471,136
1978,Climesse_Tab,-0.942029,69


In [121]:
script_growth = pd.concat([result.head(50) , result.tail(50)])
script_growth

,bnf_name,growth_rate,count
1467,Butec_Transdermal Patch 5mcg/hr,3.467742,62
1465,Butec_Transdermal Patch 10mcg/hr,3.000000,69
4415,Fostair NEXThaler_Inh 200mcg/6mcg (120D),1.430233,86
10205,Sytron_Elix 190mg/5ml S/F,1.280130,307
8498,Pneumococcal_Vac 0.5ml Vl (23 Valent),1.269430,193
...,...,...,...
5230,Hydroxyzine HCl_Oral Soln 10mg/5ml,-0.914894,94
8007,Ovysmen_Tab,-0.925373,67
1976,Climaval_Tab 1mg,-0.926471,136
1978,Climesse_Tab,-0.942029,69


In [122]:
#script_growth = [("Butec_Transdermal Patch 5mcg\/hr", 3.4677419355, 62.0)] * 100

In [1]:
who

Interactive namespace is empty.


In [123]:
grader.score.dw__script_growth(script_growth)

Your score: 0.980


## Question 6: rare_scripts

Does a practice's prescription costs originate from routine care or from reliance on rarely prescribed treatments? Commonplace treatments can carry lower costs than rare treatments because of efficiencies in large-scale production. While some specialist practices can't help but avoid prescribing rare medicines because there are no alternatives, some practices may be prescribing a unnecessary amount of brand-name products when generics are available. Let's identify practices whose costs disproportionately originate from rarely prescribed items.

First we have to identify which `'bnf_code'` are rare. To do this, find the probability $p$ of a prescription having a particular `'bnf_code'` if the `'bnf_code'` was randomly chosen from the unique options in the beneficiary data. We will call a `'bnf_code'` rare if it is prescribed at a rate less than $0.1p$.

In [8]:
# We will be calculting the probablity of any bnf_name
p = 1 / scripts['bnf_code'].nunique()
p

0.0005063291139240507

In [10]:
# Here we are calcultaing the rate of prescription for each drug....
# rate =  number of prescriptions of that bnf_code / total number of prescriptions
rate = scripts['bnf_code'].value_counts() / scripts.shape[0]
rate

0906040G0    0.014840
090402000    0.011016
0302000N0    0.010214
130201000    0.008839
0601060D0    0.008799
               ...   
0702020T0    0.000001
0902021N0    0.000001
131101000    0.000001
0501090Q0    0.000001
224041040    0.000001
Name: bnf_code, Length: 1975, dtype: float64

In [20]:
#rare_code is one whose probability is 10 times smaller than then p
rare_codes = rate < 0.1 * p

In [24]:
scripts = scripts.set_index('bnf_code').assign(rare=rare_codes).reset_index() # We have played a game here, we had first set_index to 'bnf_code' and then made the rare_codes entried in the column accroding to that and when that done we had reset_index to normal.
scripts

,bnf_code,practice,bnf_name,items,nic,act_cost,quantity,rare
0,0106020C0,N85639,Bisacodyl_Tab E/C 5mg,1,0.39,0.47,12,False
1,0106040M0,N85639,Movicol Plain_Paed Pdr Sach 6.9g,1,4.38,4.07,30,False
2,0301011R0,N85639,Salbutamol_Inha 100mcg (200 D) CFF,1,1.50,1.40,1,False
3,0304010G0,N85639,Chlorphenamine Mal_Oral Soln 2mg/5ml,1,2.62,2.44,150,False
4,0401020K0,N85639,Diazepam_Tab 2mg,1,0.16,0.26,6,False
...,...,...,...,...,...,...,...,...
973188,239410100,H81615,Coloplast_SenSura Mio Flex B/Plt 50mm C/,1,78.92,73.06,20,False
973189,239410100,H81615,Coloplast_SenSura Mio Flex Clsd Bag Mini,1,106.36,98.46,60,False
973190,239410100,H81615,Coloplast_SenSura Mio Flex Clsd Bag Midi,1,159.54,147.69,90,False
973191,239607096,H81615,Dansac_Nova 1 Convex Urost Pouch Clr C/F,2,329.76,305.28,60,False


Now for each practice, calculate the proportion of costs that originate from prescription of rare treatments (i.e. rare `'bnf_code'`). Use the `'act_cost'` field for this calculation.

In [28]:
# rare cost of all prescriptions of a given practice / cost of all prescriptions for a given practice
all_prescptions_cost = scripts.groupby('practice')['act_cost'].sum()
all_prescptions_cost

practice
A81005    103840.82
A81007    113482.49
A81011    159507.03
A81012     83296.81
A81017    232656.17
            ...    
Y05570        86.76
Y05583      1610.16
Y05597        90.41
Y05660      8762.51
Y05670        36.95
Name: act_cost, Length: 856, dtype: float64

In [31]:
rare_prescptions_cost = scripts.query('rare == True').groupby('practice')['act_cost'].sum()
rare_prescptions_cost

practice
A81005    1247.83
A81007     951.06
A81011     816.02
A81012    1145.11
A81017    1712.15
           ...   
Y05434    5918.27
Y05435    2428.19
Y05438     762.02
Y05493      40.89
Y05660       2.83
Name: act_cost, Length: 766, dtype: float64

In [35]:
rare_cost_prop = rare_prescptions_cost / all_prescptions_cost
rare_cost_prop = rare_cost_prop.fillna(0)

Now we will calculate a z-score for each practice based on this proportion.
First take the difference of `rare_cost_prop` and the proportion of costs originating from rare treatments across all practices.

In [43]:
relative_rare_cost_prop = rare_cost_prop - scripts.query('rare == True')['act_cost'].sum() /scripts['act_cost'].sum()

Now we will estimate the standard errors (i.e. the denominator of the z-score) by simply taking the standard deviation of this difference.

In [44]:
standard_errors = relative_rare_cost_prop.std()

Finally compute the z-scores. Return the practices with the top 100 z-scores in the form `(practice_code, practice_name, z-score)`. Note that some practice codes will correspond with multiple names. In this case, use the first match when sorting names alphabetically.

In [66]:
rare_scores = relative_rare_cost_prop / standard_errors
rare_scores

practice
A81005   -0.065216
A81007   -0.125308
A81011   -0.179263
A81012   -0.036615
A81017   -0.142190
            ...   
Y05570   -0.263811
Y05583   -0.263811
Y05597   -0.263811
Y05660   -0.258473
Y05670   -0.263811
Name: act_cost, Length: 856, dtype: float64

In [65]:
practice_names = (
    practices
    .sort_values('name')
    .drop_duplicates('code', keep='first')
    .set_index('code')['name']
)
practice_names


code
P87657    (IRLAM) SALFORD CARE CTRS MEDICAL PRACTI
Y05381          0-19 EAST CHESHIRE HEALTH VISITORS
Y04082       0-19 PUBLIC HEALTH SERVICE HARTLEPOOL
Y00364                               1 TO 1 CENTRE
P87652          1/LOWER BROUGHTON MEDICAL PRACTICE
                            ...                   
M83013                                      YOXALL
Y05235                   YPS SSM SERVICE (REFRESH)
E84653                         ZAIN MEDICAL CENTRE
P92005                                       ZAMAN
A81048                    ZETLAND MEDICAL PRACTICE
Name: name, Length: 10843, dtype: object

In [70]:
result = (practice_names.to_frame().assign(rare_scores=rare_scores).dropna().reset_index().nlargest(100,'rare_scores'))
result

,code,name,rare_scores
170,Y03472,CONSULTANT DIABETES TEAM,16.262687
201,Y05320,DMC COMMUNITY DERMATOLOGY RBWF,15.128648
544,Y04404,OUTPATIENTS JUBILEE HEALTH CENTRE,7.542139
200,Y03484,DMC COMMUNITY DERMATOLOGY CLINIC,7.287222
202,Y04424,DMC HEALTHCARE,6.838614
...,...,...,...
759,Y02671,THE PRACTICE HEART OF HOUNSLOW,0.208809
255,F84631,DR PI ABIOLA,0.205391
213,B81012,DR AP KUMAR,0.203691
106,C83613,CAISTOR HEALTH CENTRE,0.203500


In [ ]:
rare_scripts = [("Y03472", "CONSULTANT DIABETES TEAM", 16.2626871247)] * 100

In [71]:
grader.score.dw__rare_scripts(result)

Your score: 1.000


*Copyright &copy; 2021 WorldQuant University. This content is licensed solely for personal use. Redistribution or publication of this material is strictly prohibited.*